# Pixel drill using interactive widgets <img align="right" src="../resources/csiro_easi_logo.png">

__What does this notebook do?__

This notebook demonstrates how to perform a pixel drill using an interactive widget. A plot is drawn up of a single time slice so the user can select a pixel to interrogate further. Once a pixel is selected, a timeseries is plotted up for that pixel. Users can then select a point in the time series plot to generate a plot showing the corresponding satellite image for that point in time.

This notebook is adapted from a Digital Earth Australia example by Claire Krause

<div class="alert alert-danger">
    <strong>IMPORTANT:</strong> This notebook is still a work in progress and there are some bugs. It should work but please take note of the warning and information boxes.
</div>

In [ ]:
# %matplotlib widget

In [ ]:
# import dask
# import distributed

### EASI tools
import sys
import os
os.environ['USE_PYGEOS'] = '0' # Force the use of Shapely in Geopandas instead of Pygeos
sys.path.append('../scripts')
from app_utils import display_map
from notebook_utils import initialize_dask, localcluster_dashboard
from easi_tools import EasiDefaults
easi = EasiDefaults()

In [ ]:
cluster, client = initialize_dask(use_gateway=False)
display(client)
display(cluster)

In [ ]:
# Show the Dask dashboard - click on this to watch how the calculations are progressing
print(localcluster_dashboard(client, server=easi.hub))

#### AWS configuration
We will be using data in public requester-pays buckets, so the following configuration is required:

In [ ]:
"""This function obtains credentials for S3 access and passes them on to
   processing threads, either local or on dask cluster.
   Note that AWS credentials may need to be renewed between sessions or
   after a period of time."""

from datacube.utils.aws import configure_s3_access
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

# If not using a dask cluster then remove 'client':
# configure_s3_access(aws_unsigned=False, requester_pays=True)

In [ ]:
from datacube import Datacube
from datacube.utils import masking
import numpy as np
from datetime import datetime
import holoviews as hv
from holoviews import streams
from holoviews import opts
import warnings
warnings.filterwarnings("ignore")

# Import widgets for interactive notebook
import panel as pn
pn.extension()
pn.extension(loading_spinner='dots', loading_color='#00aa41', sizing_mode="fixed")
hv.extension('bokeh')

## Set up the extraction query

In [ ]:
# This configuration is read from the defaults for this system. 
# Examples are provided in a commented line to show how to set these manually.

study_area_lat = easi.latitude
# study_area_lat = (39.2, 39.3)

study_area_lon = easi.longitude
# study_area_lon = (-76.7, -76.6)

# product = easi.product('landsat')
# product = 'landsat8_c2l2_sr'
products = ['landsat7_c2l2_sr','landsat8_c2l2_sr','landsat9_c2l2_sr']

set_time = easi.time
# set_time = ('2020-08-01', '2020-12-01')

set_crs = easi.crs('landsat')
# set_crs = 'EPSG:32618'

set_resolution = easi.resolution('landsat')
# set_resolution = (-30, 30)

measurements = ['swir1', 'nir', 'green']

In [ ]:
dc = Datacube(app='pixel_drill')

## Plot an image with default values so that we can select a pixel for further interrogation

#### *Widgets are also provided to easily view any other region or dates in the desired time frame*

In [ ]:
# Setup the select Widgets
prod_widget = pn.widgets.Select(name='Product', 
                                options=products)

# Setup the slider Widgets 
# lat_widget = pn.widgets.RangeSlider(name='Latitude Range', start=study_area_lat[0], 
#                                     end=study_area_lat[1], value=(study_area_lat[0],study_area_lat[1]), step=0.01)
# lon_widget = pn.widgets.RangeSlider(name='Longitude Range', start=study_area_lon[0], 
#                                     end=study_area_lon[1], value=(study_area_lon[0],study_area_lon[1]), step=0.01)

# Setup the date Selection Widgets
start_date = datetime.strptime(set_time[0],'%Y-%m-%d')
start_date_widget = pn.widgets.DatePicker(name='Start Date', value=start_date.date())
end_date = datetime.strptime(set_time[1],'%Y-%m-%d')
end_date_widget = pn.widgets.DatePicker(name='End Date', value=end_date.date())

# Set up the function to return the clicked points on the image 
points = []

def record_taps(x, y):
    # display(f"{x} {y}",display_id="debug")
    if 0 not in [x,y]:
        points.clear()
        points.append((x, y, 1))
    return hv.Points(points)

# Function to get the retrieve the plots, based on the panel inputs
@pn.depends(prod_widget.param.value,
            # lat_widget.param.value, lon_widget.param.value,
            start_date_widget.param.value, end_date_widget.param.value)
def get_input_data(product, #lat, lon,
                   start_date, end_date):
    global posxy
    global data
              
    # Connect to Datacube
    dc = Datacube(app="pixel_drill")
    
    # Get attributes for loading data
    query = {'lat': (study_area_lat[0], study_area_lat[1]),
             'lon': (study_area_lon[0], study_area_lon[1]),
             'time':(start_date, end_date),
             'measurements' : measurements,
             'output_crs': set_crs,
             'resolution': set_resolution,
             'dask_chunks': {'time':1},
             'skip_broken_datasets': True}
    
    # Load data using Datacube api
    
    data = dc.load(product=product, group_by='solar_day', **query)

    # Set all nodata pixels to `NaN`:
    data = masking.mask_invalid_data(data)
    
    # Get data required for visualization and normalize it
    swir1 = data.isel(time=0).swir1.values
    nir = data.isel(time=0).nir.values
    green = data.isel(time=0).green.values

    swir1 = swir1/np.nanmax(swir1)
    nir = nir/np.nanmax(nir)
    green = green/np.nanmax(green)
    
    xmin, xmax = float(data.x.min().values), float(data.x.max().values)
    ymin, ymax = float(data.y.min().values), float(data.y.max().values)

    rgb_plot = hv.RGB(np.dstack([swir1,nir,green]), bounds=(xmin, ymin, xmax, ymax)).opts(frame_width=600, aspect='equal')
    
    # Declare SingleTap stream to capture user clicks and store the selected pixel values
    # Clicks (taps) are converted to points and added to a DynamicMap object, which is returned with the rgb_plot
    posxy = streams.SingleTap(x=0,y=0)
    
    # Create the DynamicMap which will show the clicked point
    points_dmap = hv.DynamicMap(record_taps, streams=[posxy])
    points_dmap.redim.range(x=(data.x.min().values,data.x.max().values),y=(data.y.min().values,data.y.max().values))
    points_dmap.opts(color="red", size=10)
    
    return rgb_plot * points_dmap

<div class="alert alert-info">
    Note that in Landsat 7 images, you may see that the data has black lines across it. This is due to a failure of the Scan Line Corrector (SLC) in the Landsat 7 satellite in May 2003. The result is a loss of around 22% of pixels, although the remaining data is high quality. There are various methods to fill these gaps, but none are perfect. For more information, see the Landsat 7 Scan Line Corrector (SLC) Failure section at <a href="https://www.usgs.gov/landsat-missions/landsat-7">https://www.usgs.gov/landsat-missions/landsat-7</a>
    </div>

<div class="alert alert-danger">
    <strong>IMPORTANT:</strong> This next cell might need to be run <strong>TWICE</strong> as there is something wrong in the function above. You should get a red dot appear on the map when you click to select a point. If you don't see a red dot, run this cell again and then try to click on the image.
</div>

In [ ]:
# TODO: Look into adding a loading spinner. When you change the options below, new data will load but it might take a few seconds to do so.

layout = pn.Column(
          pn.pane.Markdown('### Adjust the controls below to modify the area of interest and date range'),
          pn.Row(prod_widget),
          # pn.Row(pn.Row(pn.WidgetBox(lat_widget), pn.WidgetBox(lon_widget))),
          # pn.pane.Markdown('___Note that the date range will be used in the next cell___'),
          pn.Row(pn.Row(start_date_widget,end_date_widget)),
          # pn.pane.Markdown('### Click on the pixel you would like to interrogate and then continue to the next cell'),
          get_input_data)
layout

## Plot up the complete timeseries of data for a single pixel

In [ ]:
hv.opts("Scatter [width=600 height=600, tools=['hover']]")

green = list(data.green.sel(y=posxy.contents['y'], x=posxy.contents['x'], method='nearest').values)
swir1 = list(data.swir1.sel(y=posxy.contents['y'], x=posxy.contents['x'], method='nearest').values)
nir = list(data.nir.sel(y=posxy.contents['y'], x=posxy.contents['x'], method='nearest').values)
dates = list(data.time.values)

extents = (np.min(dates) - np.timedelta64(1,'D'), np.nanmin([green,swir1,nir]) - 20, np.max(dates) + np.timedelta64(1,'D'), np.nanmax([green,swir1,nir]) + 20)

scatter_green = hv.Scatter((dates, green), 'Date', 'Reflectance (x 10000)', extents=extents).opts(
          size=10, fill_color="green", color="green", width=800, height=400, title="Select a data point to plot up the accompanying satellite image")

scatter_swir1 = hv.Scatter((dates, swir1), 'Date', 'Reflectance (x 10000)', extents=extents).opts(
          size=10, fill_color="orange", color="orange", width=800, height=400)

scatter_nir = hv.Scatter((dates, nir), 'Date', 'Reflectance (x 10000)', extents=extents).opts(
          size=10, fill_color="red", color="red", width=800, height=400)

# Declare Tap stream with scatter as source and initial values
timeOfInterest = hv.streams.Tap(x=0, y=0)

points_green = [(x,y) for x,y in zip(dates, green)]
line_green = hv.Curve(points_green, label='green').opts(color='green')
points_swir1 = [(x,y) for x,y in zip(dates, swir1)]
line_swir1 = hv.Curve(points_swir1, label='swir1').opts(color='orange')
points_nir = [(x,y) for x,y in zip(dates, nir)]
line_nir = hv.Curve(points_nir, label='nir').opts(color='red')

points = []

def taps(x, y):

    if 0 not in [x,y]:
        points.clear()
        points.append((x, y, 1))
    return hv.Points(points)

selected_dmap = hv.DynamicMap(taps, streams=[timeOfInterest]).redim.range(x=(extents[0],extents[2]),y=(extents[1],extents[3]))
selected_dmap.opts(color="black", size=10)

list_of_curves = [
    line_nir,
    scatter_nir,
    line_swir1,
    scatter_swir1,
    line_green, 
    scatter_green
]

hv.Overlay(list_of_curves)*selected_dmap

<div class="alert alert-danger">
    <strong>IMPORTANT:</strong> In the image above, the black dot is where you have clicked. The closest date to your selection will be used for the cell below. This needs to be updated to "snap" to the points in the graph.
</div>

<div class="alert alert-info">

**Note:** In this example, we have not filtered out clouds. You can see in the above plot that there are some dates with high and low values for reflectance. If you click through these pixels, you will notice that these higher values are actually images where the scene is cloudy - where the reflectance from cloud is very high. The lower values represent the scenes where the sensor had a clear image of the pixel chosen, and so represent actual ground green band values. If we had filtered out the cloud impacted scenes, the scenes with higher values would be removed, as would the values below zero, which represent missing values for the chosen pixel.

</div>

## Plot the scene that matches the selected time and sensor

In [ ]:
image_array = data[['swir1', 'nir', 'green']].sel(time=timeOfInterest.contents['x'],
                                                  method='nearest')
date = datetime.strptime(str(image_array.time.values),'%Y-%m-%dT%H:%M:%S.%f000')

swir1_time_of_interest = image_array.swir1.values
nir_time_of_interest = image_array.nir.values
green_time_of_interest = image_array.green.values

swir1_time_of_interest = swir1_time_of_interest/np.nanmax(swir1_time_of_interest)
nir_time_of_interest = nir_time_of_interest/np.nanmax(nir_time_of_interest)
green_time_of_interest = green_time_of_interest/np.nanmax(green_time_of_interest)

xmin_img_arr, xmax_img_arr = float(image_array.x.min().values), float(image_array.x.max().values)
ymin_img_arr, ymax_img_arr = float(image_array.y.min().values), float(image_array.y.max().values)

hv.RGB(np.dstack([swir1_time_of_interest, nir_time_of_interest, green_time_of_interest]),
       bounds=(xmin_img_arr, ymin_img_arr, xmax_img_arr, ymax_img_arr)).opts(frame_width=600, aspect='equal', title=f'{prod_widget.value} scene for {date.strftime("%d %B %Y")}')

In [ ]:
client.close()
cluster.close()